### Example 1

In [1]:
using Gurobi

env = Gurobi.Env()

# set presolve to 0
setparam!(env, "Presolve", 0)

 # construct the model
model = gurobi_model(env;
    name = "lp_01",
    f = ones(2),
    A = [50. 24.; 30. 33.],
    b = [2400., 2100.],
    lb = [5., 45.])

 # run optimization
optimize(model)

 # show results
sol = get_solution(model)
println("soln = $(sol)")

objv = get_objval(model)
println("objv = $(objv)")

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [2e+01, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 5e+01]
  RHS range        [2e+03, 2e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  5.000000000e+01
soln = [5.0,45.0]
objv = 50.0


### Example 2

In [3]:
using Gurobi

env = Gurobi.Env()

# set presolve to 0
setparam!(env, "Presolve", 0)

 # creates an empty model ("lp_01" is the model name)
model = Gurobi.Model(env, "lp_01", :maximize)

 # add variables
 # add_cvar!(model, obj_coef, lower_bound, upper_bound)
add_cvar!(model, 1.0, 45., Inf)  # x: x >= 45
add_cvar!(model, 1.0,  5., Inf)  # y: y >= 5

 # For Gurobi, you have to call update_model to have the
 # lastest changes take effect
update_model!(model)

 # add constraints
 # add_constr!(model, coefs, sense, rhs)
add_constr!(model, [50., 24.], '<', 2400.) # 50 x + 24 y <= 2400
add_constr!(model, [30., 33.], '<', 2100.) # 30 x + 33 y <= 2100
update_model!(model)

println(model)

 # perform optimization
optimize(model)

Gurobi Model: lp_01
    type   : LP
    sense  : maximize
    number of variables             = 2
    number of linear constraints    = 2
    number of quadratic constraints = 0
    number of sos constraints       = 0
    number of non-zero coeffs       = 4
    number of non-zero qp objective terms  = 0
    number of non-zero qp constraint terms = 0

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [2e+01, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 5e+01]
  RHS range        [2e+03, 2e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+30   2.140625e+30   2.000000e+00      0s
       1    5.1250000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds
Optimal objective  5.125000000e+01


### Example 3

In [5]:
using JuMP, Gurobi

# pass params as keyword arguments to GurobiSolver
m = Model(solver=GurobiSolver(Presolve=0))

@variable(m, x >= 5)
@variable(m, y >= 45)

@objective(m, Min, x + y)
@constraint(m, 50x + 24y <= 2400)
@constraint(m, 30x + 33y <= 2100)

status = solve(m)
println("Optimal objective: ",getobjectivevalue(m),
	". x = ", getvalue(x), " y = ", getvalue(y))

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [2e+01, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 5e+01]
  RHS range        [2e+03, 2e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  5.000000000e+01
Optimal objective: 50.0. x = 5.0 y = 45.0


### Example 4 - High level quadratic equation

In [8]:
# The problem formulation is
# minimize x^2 + xy + y^2 + yz + z^2
# s.t.  x + 2 y + 3 z >= 4
#       x +   y       >= 1

# Example 2.1: High-level Quadratic Programming API
#using the function gurobi_model:

using Gurobi

env = Gurobi.Env()

model = gurobi_model(env;
        name = "qp_01",
        H = [2. 1. 0.; 1. 2. 1.; 0. 1. 2.],
        f = [0., 0., 0.],
        A = -[1. 2. 3.; 1. 1. 0.],
        b = -[4., 1.])
optimize(model)

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model has 5 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 5 nonzeros
Presolved model has 5 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 5
 AA' NZ     : 6.000e+00
 Factor NZ  : 1.000e+01
 Factor Ops : 3.000e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.75823538e-13 -4.75823538e-13  2.00e+00 4.21e-07  9.98e+05     0s
   1   1.44987169e+05 -1.43989081e+05  8.42e-08 1.97e-04  1.44e+05     0s
   2   2.13605434e+04 -2.09773620e+04  4.71e-08 1.21e-04  2.12e+04     0s
   3   3.17522703e+03 -3.02742005e+03  3.62e-09 8.83e-06  3.10e+03     0s

### Example 5 - Mixed Integer programming

In [ ]:
# Problem formulation:

# maximize x + 2 y + 5 z

# s.t.  x + y + z <= 10
#      x + 2 y + z <= 15
#      x is continuous: 0 <= x <= 5
#      y is integer: 0 <= y <= 10
#      z is binary

In [2]:
using Gurobi

env = Gurobi.Env()

model = Gurobi.Model(env, "mip_01", :maximize)

 # add continuous variable
add_cvar!(model, 1., 0., 5.)  # x

 # add integer variable
add_ivar!(model, 2., 0, 10)   # y

 # add binary variable
add_bvar!(model, 5.)          # z

 # have the variables incorporated into the model
update_model!(model)

add_constr!(model, ones(3), '<', 10.)
add_constr!(model, [1., 2., 1.], '<', 15.)

optimize(model)

Optimize a model with 2 rows, 3 columns and 6 nonzeros
Variable types: 1 continuous, 2 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+01, 2e+01]
Found heuristic solution: objective 15
Presolve removed 0 rows and 1 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: objective 1.900000e+01, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      19.0000000   19.00000  0.00%     -    0s

Explored 0 nodes (2 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 19 15 
Pool objective bound 19

Optimal solution found (tolerance 1.00e-04)
Best objective 1.900000000000e+01, best bound 1.9

### Same problem using JuMP

In [11]:
using JuMP, Gurobi

m = Model(solver=GurobiSolver())

@variables(m,begin
    0 <= x <= 5
    0 <= y <= 10, Int
    z, Bin
end)

@objective(m, Max, x + 2y + 5z)
@constraint(m, x + y + z <= 10)
@constraint(m, x + 2y + z <= 15)

solve(m)

:Optimal

Optimize a model with 2 rows, 3 columns and 6 nonzeros
Variable types: 1 continuous, 2 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+01, 2e+01]
Found heuristic solution: objective 15
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: objective 1.900000e+01, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      19.0000000   19.00000  0.00%     -    0s

Explored 0 nodes (2 simplex iterations) in 0.00 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 19 15 
Pool objective bound 19

Optimal solution found (tolerance 1.00e-04)
Best objective 1.900000000000e+01, best bound 1.9